In [2]:
# Import components
import datetime as dt
import math
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pmdarima as pm
import seaborn as sns
from matplotlib import pyplot as pltw
from pmdarima import arima, datasets, model_selection, utils
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import LocalOutlierFactor
from statsmodels.tsa.seasonal import STL

warnings.simplefilter("ignore")

plt.rcParams["font.family"] = "Hiragino Maru Gothic Pro"
plt.style.use("ggplot")  # グラフのスタイル
plt.rcParams["figure.figsize"] = [12, 9]  # グラフサイズ設定

In [3]:
df = pd.read_csv("../datasets/fixed_battery_log_2.csv")
df["date"] = pd.to_datetime(df["date"])
users = df["User"].unique()

In [27]:
data_nari = df.loc[(df["User"] == "Narita") & (df["state"] == "ON")].reset_index()
data_nari.tail()

,index,User,date,battery,state,weekday,time_to_num
275,550,Narita,2022-03-31 07:50:00,61,ON,3,28242
276,552,Narita,2022-03-31 09:27:00,75,ON,3,34058
277,554,Narita,2022-03-31 10:02:00,73,ON,3,36131
278,556,Narita,2022-03-31 11:51:00,86,ON,3,42683
279,558,Narita,2022-03-31 16:34:00,57,ON,3,59675


In [28]:
data_nari.loc[:]["battery"]

0      60
1      71
2      73
3      30
4      42
       ..
275    61
276    75
277    73
278    86
279    57
Name: battery, Length: 280, dtype: int64

In [26]:
q = data_nari.battery.quantile(0.95)
dn_q_25 = data_nari.query("battery < @q").reset_index()
del dn_q_25["level_0"]
dn_q_25.tail()

,index,User,date,battery,state,weekday,time_to_num
259,550,Narita,2022-03-31 07:50:00,61,ON,3,28242
260,552,Narita,2022-03-31 09:27:00,75,ON,3,34058
261,554,Narita,2022-03-31 10:02:00,73,ON,3,36131
262,556,Narita,2022-03-31 11:51:00,86,ON,3,42683
263,558,Narita,2022-03-31 16:34:00,57,ON,3,59675


In [29]:
clf = LocalOutlierFactor(n_neighbors=5)